In [ ]:
import bw2data as bd
import bw2regional as bwr
from pathlib import Path

In [ ]:
PROJECT_NAME = "Spain case study"

In [ ]:
RESET = True

In [ ]:
if PROJECT_NAME in bd.projects and RESET:
    bd.projects.delete_project(PROJECT_NAME, True) 

In [ ]:
bd.projects.set_current(PROJECT_NAME)

In [ ]:
data_dir = Path.cwd().absolute() / "data"
assert data_dir.is_dir()

Tell `bw2regional` about our maps.

Data from Natural Earth Data. Needed to create another column named `city_id` with a string data type.

In [ ]:
bwr.geocollections['cities'] = {
    'filepath': str(data_dir / 'cities.gpkg'),
    'field': 'city_id',
}

In [ ]:
bwr.geocollections['regions'] = {
    'filepath': str(data_dir / 'regions.gpkg'),
    'field': 'name',
}

In [ ]:
bwr.geocollections['countries'] = {
    'filepath': str(data_dir / 'countries.gpkg'),
    'field': 'NAME',
}

In [ ]:
bwr.geocollections['popdensity'] = {'filepath': str(data_dir / 'gpw_v4_population_density.tif')}

CF map from [WRI](https://www.wri.org/data/aqueduct-water-stress-projections-data)

In [ ]:
bwr.geocollections['water-stress'] = {
    'filepath': str(data_dir / 'spain_water_stress.gpkg'),
    'field': 'BasinID',
}

Data from [Estimation of spatial distribution of irrigated crop areas in Europe for large-scale modelling applications](https://www.sciencedirect.com/science/article/pii/S0378377422000749?via%3Dihub#sec0060), via [Agri4Cast](https://agri4cast.jrc.ec.europa.eu/DataPortal/Index.aspx?o=).

Data also [available here](https://files.brightway.dev/europe_irrigated.gpkg).

GDAL commands to extract and process the rasters:

```bash
gdal_rasterize -a IR_citrus -init -999 -a_nodata -999 -ts 400 400 -of GTIFF irrigation.gpkg citrus.p.tiff
gdal_rasterize -a IR_potatoe -init -999 -a_nodata -999 -ts 400 400 -of GTIFF irrigation.gpkg potatoe.p.tiff
gdal_rasterize -a IR_rice -init -999 -a_nodata -999 -ts 400 400 -of GTIFF irrigation.gpkg rice.p.tiff
gdal_rasterize -a IR_cereals -init -999 -a_nodata -999 -ts 400 400 -of GTIFF irrigation.gpkg cereals.p.tiff

gdalwarp -t_srs EPSG:4326 rice.p.tiff rice.tiff
gdalwarp -t_srs EPSG:4326 potatoe.p.tiff potatoe.tiff
gdalwarp -t_srs EPSG:4326 cereals.p.tiff cereals.tiff
gdalwarp -t_srs EPSG:4326 citrus.p.tiff citrus.tiff
```

In [ ]:
CROPS = ['cereals', 'citrus', 'rice', 'potatoe', 'sunflowers']

In [ ]:
for crop in CROPS:
    bwr.geocollections[crop] = {'filepath': str(data_dir / f'{crop}.tiff'), 'nodata': -999}

In [ ]:
bio = bd.Database("biosphere")
bio.register()

In [ ]:
h2o = bio.new_node(
    code="water",
    name="water",
    type="emission",
)
h2o.save()

In [ ]:
co2 = bio.new_node(
    code="co2",
    name="co2",
    type="emission",
)
co2.save()

In [ ]:
ch4 = bio.new_node(
    code="ch4",
    name="ch4",
    type="emission",
)
ch4.save()

In [ ]:
h2o = bd.get_node(name="water")

# Insert your own meal here

In [ ]:
food = bd.Database("food")
food.register()

lemon = food.new_node(
    code="lemon",
    name="lemon",
    location=('regions', 'Granada')
)
lemon.save()
lemon.new_edge(
    input=h2o,
    amount=5,
    type="biosphere",
).save()

mushroom = food.new_node(
    code="mushroom",
    name="mushroom",
    location=('countries', 'Portugal')
)
mushroom.save()
mushroom.new_edge(
    input=h2o,
    amount=0.5,
    type="biosphere",
).save()

cheese = food.new_node(
    code="cheese",
    name="cheese",
    location=('countries', 'Spain')
)
cheese.save()
cheese.new_edge(
    input=h2o,
    amount=25,
    type="biosphere",
).save()

rice = food.new_node(
    code="rice",
    name="rice",
    location=('regions', 'Valencia')
)
rice.save()
rice.new_edge(
    input=h2o,
    amount=10,
    type="biosphere",
).save()

meal = food.new_node(
    code="meal",
    name="meal",
    location=('cities', '14')
)
meal.save()
meal.new_edge(
    input=h2o,
    amount=0.5,
    type="biosphere",
).save()
meal.new_edge(
    input=lemon,
    amount=0.25,
    type="technosphere",
).save()
meal.new_edge(
    input=rice,
    amount=1,
    type="technosphere",
).save()
meal.new_edge(
    input=mushroom,
    amount=0.5,
    type="technosphere",
).save()

meal.new_edge(
    input=cheese,
    amount=0.1,
    type="technosphere",
).save()

In [ ]:
bio.set_geocollections()
food.set_geocollections()

In [ ]:
import fiona
from shapely.geometry import shape
from collections import defaultdict
import numpy as np

In [ ]:
water_stress = bd.Method(("water stress",))
water_stress.register()

In [ ]:
bwr.import_regionalized_cfs(
    'water-stress',
    ("water stress",),
    {"ws2038cr": [h2o.key]},
    nan_value=-1,
)

In [ ]:
for place in ("cities", "regions", "countries"):
    if f'{place}-water-stress' not in bwr.geocollections:
        bwr.remote.calculate_intersection(place, 'water-stress')
        bwr.remote.intersection_as_new_geocollection(place, 'water-stress', f'{place}-water-stress')

In [ ]:
for crop in ('cereals', 'citrus', 'rice', 'potatoe', 'popdensity'):
    if f'cities-water-stress - {crop}' not in bwr.extension_tables:
        bwr.raster_as_extension_table('cities-water-stress', crop)
    if f'regions-water-stress - {crop}' not in bwr.extension_tables:
        bwr.raster_as_extension_table('regions-water-stress', crop)
    if f'countries-water-stress - {crop}' not in bwr.extension_tables:
        bwr.raster_as_extension_table('countries-water-stress', crop)        

In [ ]:
bwr.calculate_needed_intersections({meal: 1}, ("water stress",))

for xt in bwr.extension_tables:
    bwr.calculate_needed_intersections({meal: 1}, ("water stress",), xt)

In [ ]:
bwr.extension_tables

In [ ]:
lca_configuration = [
    # {
    #     'xts': ['cities-water-stress - cereals', 'countries-water-stress - cereals', 'regions-water-stress - cereals'],
    #     'nodes': [],
    # },
    {
        'xts': ['cities-water-stress - rice', 'countries-water-stress - rice', 'regions-water-stress - rice'],
        'nodes': [bd.get_node(name='rice').id],
    },
    {
        'xts': ['cities-water-stress - citrus', 'countries-water-stress - citrus', 'regions-water-stress - citrus'],
        'nodes': [bd.get_node(name='lemon').id],
    },
    # {
    #     'xts': ['cities-water-stress - potatoe', 'countries-water-stress - potatoe', 'regions-water-stress - potatoe'],
    #     'nodes': [],
    # },
    {
        'xts': ['cities-water-stress - popdensity', 'countries-water-stress - popdensity', 'regions-water-stress - popdensity'],
        'nodes': [bd.get_node(name='mushroom').id, bd.get_node(name='meal').id, bd.get_node(name='cheese').id],
    },
]

In [ ]:
characterized_inventory = None

for dct in lca_configuration:
    for xt in dct['xts']:
        lca = bwr.ExtensionTablesLCA(
            demand={meal: 1},
            method=("water stress",),
            xtable=xt,
            limitations={'activities': dct['nodes'], 'activities mode': 'include'},
        )
        lca.lci()
        lca.lcia()
        if characterized_inventory is None:
            characterized_inventory = lca.characterized_inventory
        else:
            characterized_inventory += lca.characterized_inventory

In [ ]:
lca.geodataframe_xtable_spatial_scale().explore(column='score_rel')